In [1]:
#import support lib
import json
import os
from PIL import Image, ImageOps
import pydicom
import numpy as np
import shutil
from collections import OrderedDict
import cv2
from threading import Thread
import math
from random import shuffle

In [2]:
from Database.Sqlite3 import DB_SQLITE3
from random import shuffle
import os

In [6]:
with open('resource/state/state.json', 'r') as json_file:  
    state_file = json.load(json_file)
    
json_file.close()

db = DB_SQLITE3("config/blindtest_ex_augment1.db")

In [4]:
normals = []
milds = []
moderates = []
severes = []
others = []

for fold_number in range(5):
    
    condition = '%/' + str(fold_number) + '/%'
    
    # count normal
    normal_rows = db.select_by_cond(table_name='dataset_info', dict_condition = {'path':condition, 'type': 'Normal'}, list_condition_operation = ['like', '='])
    normals.insert(fold_number, len(normal_rows))
    # count mild
    mild_rows = db.select_by_cond(table_name='dataset_info', dict_condition = {'path':condition, 'type': 'Mild'}, list_condition_operation = ['like', '='])
    milds.insert(fold_number, len(mild_rows))
    # count moderate
    moderate_rows = db.select_by_cond(table_name='dataset_info', dict_condition = {'path':condition, 'type': 'Moderate'}, list_condition_operation = ['like', '='])
    moderates.insert(fold_number, len(moderate_rows))
    # count severe
    severe_rows = db.select_by_cond(table_name='dataset_info', dict_condition = {'path':condition, 'type': 'Severe'}, list_condition_operation = ['like', '='])
    severes.insert(fold_number, len(severe_rows))
    # count other
    other_rows = db.select_by_cond(table_name='dataset_info', dict_condition = {'path':condition, 'type': 'Others'}, list_condition_operation = ['like', '='])
    others.insert(fold_number, len(other_rows))
    
print(normals)
print(milds)
print(moderates)
print(severes)
print(others)

[24840, 24840, 24840, 24840, 25920]
[19440, 19440, 19440, 19440, 19440]
[15120, 15120, 15120, 15120, 15120]
[12960, 12960, 12960, 12960, 17280]
[5400, 5400, 5400, 5400, 8640]


In [7]:
rows = db.count(table_name='dataset_info') + 1
count_normal = 0
count_mild = 0
count_moderate = 0
count_severe = 0
count_other = 0


for fold_number in range(5):
    
    condition = '%/' + str(fold_number) + '/%'
    rows = db.select_by_cond(table_name='dataset_info', dict_condition = {'path':condition}, list_condition_operation = ['like'])
    
    new_fold = []
    
    severe = severes[fold_number]
    absevere = milds[fold_number] + moderates[fold_number] + normals[fold_number] + others[fold_number]
    # calculate ratio
    mild_ratio = milds[fold_number] / absevere
    moderate_ratio = moderates[fold_number] / absevere
    normal_ratio = normals[fold_number] / absevere
    other_ratio = others[fold_number] / absevere
    # calculate number of images
    mild_nb = math.ceil(round(mild_ratio, 2) * severe)
    moderate_nb = math.ceil(round(moderate_ratio,2)*severe)
    normal_nb = math.ceil(round(normal_ratio,2)*severe)
    other_nb = math.ceil(round(other_ratio,2)*severe)
    
#     print(mild_nb)
#     print(moderate_nb)
#     print(severe_nb)
#     print(other_nb)
    
    for r in rows:
        img_id = r[0]
        level = r[2]
 
        if level == 'Severe':
            new_fold.append(img_id)
            count_severe += 1
        elif level == 'Mild' and mild_nb != 0:
            new_fold.append(img_id)
            mild_nb -= 1
            count_mild += 1
        elif level == 'Moderate' and moderate_nb != 0:
            new_fold.append(img_id)
            moderate_nb -= 1
            count_moderate += 1
        elif level == 'Normal' and normal_nb != 0:
            new_fold.append(img_id)
            normal_nb -= 1
            count_normal += 1
        elif level == 'Others' and other_nb != 0:
            new_fold.append(img_id)
            other_nb -= 1
            count_other += 1
    print('fold_number: ', fold_number)        
    print('old fold: ', len(rows))
    print('new fold: ',len(new_fold))
    print('\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\')
    
    shuffle(new_fold)
    state_file['cross_validation_id'][fold_number] = new_fold
    with open('resource/state/state_balance_severe_augment1.json', 'w') as json_file:  
        json.dump(state_file, json_file)
    json_file.close()


fold_number:  0
old fold:  77760
new fold:  25791
\\\\\\\\\\\\\\\\\\
fold_number:  1
old fold:  77760
new fold:  25791
\\\\\\\\\\\\\\\\\\
fold_number:  2
old fold:  77760
new fold:  25791
\\\\\\\\\\\\\\\\\\
fold_number:  3
old fold:  77760
new fold:  25791
\\\\\\\\\\\\\\\\\\
fold_number:  4
old fold:  86400
new fold:  34562
\\\\\\\\\\\\\\\\\\


In [8]:
print('count_normal ', count_normal)
print('count_mild ', count_mild)
print('count_moderate ', count_moderate)
print('count_severe ', count_severe)
print('count_other ', count_other)
print('count_absevere ', count_mild + count_moderate + count_normal + count_other)

count_normal  26267
count_mild  20391
count_moderate  15726
count_severe  69120
count_other  6222
count_absevere  68606


In [9]:
with open('resource/state/state.json', 'r') as json_file:  
    state_file = json.load(json_file)
    
json_file.close()

In [10]:
for index, fold in enumerate(state_file['cross_validation_id']):
    print(len(fold), end="\n\n")

49681

49681

49681

49681

51841

